# Dataset

The NSF Research Awards Abstracts dataset comprises several paper abstracts, one per file, that were furnished by the NSF (National Science Foundation).

For this especific project we used [a sample from 2020](https://www.nsf.gov/awardsearch/download?DownloadFileName=2020&All=true).

I start by creating a single CSV file with only the abstract and filename

In [5]:
import xml.etree.ElementTree as Xet 
import pandas as pd
import os

input_directory: str = os.path.join('data', 'inputs')
filepaths = next(os.walk(input_directory), (None, None, []))[2]  # [] if no file

data = []

for filepath in filepaths:
    xmlparse = Xet.parse(os.path.join(input_directory, filepath)) 
    root = xmlparse.getroot() 
    for tag in root:
        data.append({
            "file": filepath,
            "abstract": tag.find("AbstractNarration").text
        })

df = pd.DataFrame(data) 

# Writing dataframe to csv 
df.to_csv(os.path.join('data', 'processed', 'abstracts.csv'), index=False)